# Copy instead of modifying to preserve attack parameters

In [20]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SURROGATE_PATH = 'surrogates/imitator.pth'
#RUNS = 5
TRIAL = 1
SAVE_DIR = ''

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.estimators.classification import PyTorchClassifier as classifier
from art.attacks.evasion import FastGradientMethod as FGM

import pandas as pd
import numpy as np
import os
import torch
from torch.nn import CrossEntropyLoss

import KBMproject.utilities as utils

from joblib import Parallel, delayed

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
testilons = np.arange(0.02, 0.21, 0.01)
RUNS = len(testilons)

Define RL agent

In [5]:
agents = []
path = os.path.join(os.getcwd(), '..',AGENT_NAME)
path = os.path.normpath(path) #resolve '..'
for _ in range (RUNS):
    agents.append(PPO.load(path))

Create environments

In [6]:
envs = []
for _ in range (RUNS):
    envs.append(utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42))

In [7]:
cols = utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42).observation_names

In [8]:
path = os.path.join(os.getcwd(), '..',SURROGATE_PATH)
path = os.path.normpath(path) #resolve '..'
imitator = torch.load(path,)

In [9]:
path = os.path.join(os.getcwd(), '..','..','observation_masks.csv') #get adversary from parent directory
path = os.path.normpath(path) #resolve '..'
observation_masks = pd.read_csv(path)

In [11]:
kwargs = dict(norm=np.inf,
        #eps=0.05,
        #num_random_init=2,
        )
attacks =[]
for run in range(RUNS):
    attacks.append(FGM(estimator=classifier(
                                        model=imitator,
                                        loss=CrossEntropyLoss(), 
                                        nb_classes=agents[run].action_space[0].n, 
                                        input_shape=agents[run].observation_space.shape,
                                        device_type='gpu',
                                        clip_values = (agents[run].observation_space.low.min(),
                                                       agents[run].observation_space.high.max()),
                                        ),
                        eps=testilons[run],
                        **kwargs))

In [12]:
%%time
%%capture
results = Parallel(n_jobs=RUNS, verbose=10, prefer='threads')(delayed(
    utils.eval_untargeted_attack)(agent, env, attack) for agent, env, attack in zip(agents, envs, attacks)) 


CPU times: total: 36min 12s
Wall time: 21min 20s


Results is a list of tupples for each run, of the format(KPIs, observations, perturbed observations, epsilons)

In [13]:
kpis = [results[i][0] for i in range(len(results))]
df_kpis = pd.concat(kpis, axis='columns',keys=testilons)

In [14]:
#df_kpis[['mean', 'std', 'variance']] = df_kpis.agg(['mean','std', 'var'], axis='columns')

In [15]:
df_kpis

,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20
cost_function,,,,,,,,,,,,,,,,,,,
annual_peak_average,1.045359,1.125453,1.125444,1.009986,1.042875,1.042875,1.042875,1.314043,1.314043,1.120856,1.120856,1.250703,1.120856,1.120856,1.231993,1.231993,1.231993,1.214377,1.214377
carbon_emissions_total,0.876804,0.882333,0.885240,0.891395,0.898524,0.900856,0.904220,0.907534,0.911456,0.916498,0.922078,0.929926,0.934546,0.939253,0.950346,0.954268,0.959317,0.968259,0.980023
cost_total,0.794263,0.799455,0.803086,0.810279,0.817135,0.820568,0.825075,0.832173,0.837394,0.842390,0.850811,0.858732,0.864673,0.871779,0.889422,0.893528,0.897881,0.907243,0.920987
daily_one_minus_load_factor_average,1.142974,1.058087,1.045373,1.070906,0.959250,1.060939,1.058108,1.040683,1.067220,1.066344,1.071346,0.909380,1.041163,1.032757,1.030620,0.909537,0.912396,1.038642,1.024081
daily_peak_average,0.905562,0.915999,0.919343,0.939351,0.962212,0.965413,0.972757,0.984121,0.998884,1.015347,1.014155,1.048801,1.049001,1.067986,1.116411,1.136305,1.139813,1.158254,1.187960
electricity_consumption_total,0.886646,0.891857,0.893891,0.899524,0.906510,0.908677,0.911625,0.914440,0.918482,0.922794,0.928326,0.935454,0.939368,0.943980,0.952309,0.956812,0.962165,0.970943,0.980652
monthly_one_minus_load_factor_average,0.986091,0.982800,0.980234,0.991022,0.991840,0.993014,0.987186,0.986489,0.990937,1.001909,1.001768,1.005537,0.994660,1.002688,1.009864,1.008777,1.005865,1.008864,1.007598
ramping_average,1.128752,1.155390,1.186204,1.224792,1.261591,1.276214,1.295461,1.311909,1.341885,1.365701,1.386293,1.436371,1.449518,1.487045,1.543223,1.571265,1.601899,1.640265,1.676788
zero_net_energy,1.107790,1.107731,1.108618,1.109547,1.110702,1.110793,1.111426,1.111587,1.112076,1.112732,1.112917,1.114421,1.115331,1.116427,1.118495,1.119232,1.120682,1.122065,1.123619


In [16]:
df_kpis.to_csv(SAVE_DIR + f'{RUNS} run KPIs {TRIAL}.csv')

FGSM using the same parameters in a whitebox attack has an ASR of 0.7.

Increasing the number of restarts decreases the ASR, likely because of differences between the surrogate and vicitm models

In [17]:
ASRs = [results[i][3] for i in range(len(results))]
#print(f'For 5 runs of the random attack \nthe mean ASR is: {np.mean(ASRs):.3f}\nthe STD is: {np.std(ASRs):.3f}')

In [18]:
pd.DataFrame({'ASRs':ASRs}, index=testilons,).to_csv(SAVE_DIR + 'ASR for varied epsilons.csv')

In [21]:
for run in range(RUNS):
        pd.DataFrame(results[run][1],columns=cols,).to_csv(SAVE_DIR+f'run {run} obs {TRIAL}.csv',)
        pd.DataFrame(results[run][2],columns=cols,).to_csv(SAVE_DIR+f'run {run} adv obs {TRIAL}.csv')